In this file, I will be showcasing a simple implementation of how to use the model earlier trained in model.ipynb

In [1]:
import joblib
import pandas as pd
import yfinance as yf
from sklearn.preprocessing import StandardScaler
import numpy as np
import datetime

In [3]:
# Load the model
model = joblib.load('final_gbm_model.pkl')

In [5]:
# Fetch LIVE S&P 500 data
ticker = "^GSPC"
data = yf.download(ticker, start="2023-01-01", end=datetime.date.today().strftime("%Y-%m-%d"))

[*********************100%%**********************]  1 of 1 completed


In [6]:
# Calculate necessary features
data['Lag_1'] = data['Close'].shift(1)
data['Lag_2'] = data['Close'].shift(2)
data['Lag_3'] = data['Close'].shift(3)
data['Lag_4'] = data['Close'].shift(4)
data['Lag_5'] = data['Close'].shift(5)
data['Lag_6'] = data['Close'].shift(6)
data['Lag_7'] = data['Close'].shift(7)
data['Daily_Return'] = data['Close'].pct_change()
data['Lag_Return_1'] = data['Daily_Return'].shift(1)
data['Lag_Return_2'] = data['Daily_Return'].shift(2)
data['Lag_Return_3'] = data['Daily_Return'].shift(3)
data['Rolling_Std'] = data['Close'].rolling(window=30).std()
data['Rolling_Mean'] = data['Close'].rolling(window=30).mean()
data['Rolling_Median'] = data['Close'].rolling(window=30).median()
data['MA50'] = data['Close'].rolling(window=50).mean()
data['MA200'] = data['Close'].rolling(window=200).mean()

delta = data['Close'].diff()
gain = (delta.where(delta > 0, 0)).rolling(window=14).mean()
loss = (-delta.where(delta < 0, 0)).rolling(window=14).mean()
rs = gain / loss
data['RSI'] = 100 - (100 / (1 + rs))

ema12 = data['Close'].ewm(span=12, adjust=False).mean()
ema26 = data['Close'].ewm(span=26, adjust=False).mean()
data['MACD'] = ema12 - ema26

# Drop rows with NaN values created by lagging and rolling calculations
data = data.dropna()

# Define the feature set
features = ['Lag_1', 'Lag_2', 'Lag_3', 'Lag_4', 'Lag_5', 'Lag_6', 'Lag_7', 
            'Lag_Return_1', 'Lag_Return_2', 'Lag_Return_3', 'Daily_Return', 
            'Rolling_Std', 'Rolling_Mean', 'Rolling_Median', 'MA50', 'MA200', 'RSI', 'MACD']


In [9]:
# Select the latest data point for prediction
latest_data = data.iloc[-1:][features]

# Make prediction
prediction = model.predict(latest_data)

# Output the result
if prediction == 1:
    print("The model predicts that the S&P 500 price will go UP tomorrow.")
else:
    print("The model predicts that the S&P 500 price will go DOWN tomorrow.")


The model predicts that the S&P 500 price will go DOWN tomorrow.


c:\Users\boxle\OneDrive\Desktop\Stock-Market-Analysis\stockanalysis\lib\site-packages\sklearn\base.py:458: UserWarning: X has feature names, but GradientBoostingClassifier was fitted without feature names
  warnings.warn(


And there we have it, a model that predicts whether SNP500 price will increase or decrease the next day so I can know when to buy in.